In [54]:
# Supress warnings
import warnings
warnings.filterwarnings('ignore')

# Dependencies
import numpy as np
import pandas as pd
import geopandas as geopd 
import seaborn as sns
import matplotlib as mpl 
import matplotlib.pyplot as plt 

In [55]:
''' Import housing sales dataset '''
house_df = pd.read_csv('./data/Original/house_sales.csv')
house_df.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650.0,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242.0,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000.0,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000.0,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080.0,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4.0,4.50,5420.0,101930.0,1.0,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3.0,2.25,1715.0,6819.0,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3.0,1.50,1060.0,9711.0,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3.0,1.00,1780.0,7470.0,1.0,0,0,...,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3.0,2.50,1890.0,6560.0,2.0,0,0,...,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


In [56]:
''' Create new dataframe to preserve original dataset and assign to new variable '''
house_datesClean = house_df      
dates = house_datesClean["date"]
dates.head()

0    20141013T000000
1    20141209T000000
2    20150225T000000
3    20141209T000000
4    20150218T000000
Name: date, dtype: object

In [57]:
''' Strip 'T000000' from date column and insert into housing sales dataframe '''
house_datesClean['date'] = pd.Series([date.strip('T000000') for date in dates], index=house_datesClean.index)
house_datesClean['date'].head()

0    20141013
1    20141209
2    20150225
3    20141209
4    20150218
Name: date, dtype: object

In [58]:
''' Replace missing values with mean lot size per zipcode. '''
house_datesClean['sqft_lot'] = house_datesClean['sqft_lot'].fillna(house_datesClean
                                                           .groupby('zipcode')['sqft_lot15'].transform('mean'))
house_datesClean['sqft_lot']

0         5650.0
1         7242.0
2        10000.0
3         5000.0
4         8080.0
          ...   
21608     1131.0
21609     5813.0
21610     1350.0
21611     2388.0
21612     1076.0
Name: sqft_lot, Length: 21613, dtype: float64

In [59]:
''' Find missing values of sqft_living by summing the sqft_above and sqft_basement variables '''
sqft_living_total = house_datesClean['sqft_above'] + house_datesClean['sqft_basement']
house_datesClean['sqft_living'] = house_datesClean['sqft_living'].fillna(sqft_living_total, axis=0)

In [60]:
#Remove Outliers for Bedrooms and Bathrooms
house_datesClean.loc[(house_datesClean.bedrooms == 0) & (house_datesClean.bedrooms > 10), 'bedrooms'] = np.nan
house_datesClean.loc[(house_datesClean.bathrooms == 0), 'bathrooms'] = np.nan

#Determining bedroom missing values by taking the mean of the bedrooms grouped by sqft_living
bins =  np.arange(start=250, stop=12100, step=1500)
ind = np.digitize(house_datesClean['sqft_living'], bins)
bed_mean = round(house_datesClean.groupby(ind)['bedrooms'].transform('mean').sort_index(),0)

house_datesClean['bedrooms'] = house_datesClean['bedrooms'].fillna(bed_mean)

#Determining bathroom missing values by taking the mean of bathrooms grouped by sqft_living
bath_mean = round(house_datesClean.groupby(ind)['bathrooms'].transform('mean').sort_index(),3)
house_datesClean['bathrooms'] = house_datesClean['bathrooms'].fillna(bath_mean)

In [61]:
''' Display head of cleaned dataframe and save to csv '''
houseCleanDF = house_datesClean
houseCleanDF.to_csv('./data/Modified/houseClean.csv')
houseCleanDF.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013,221900.0,3.0,1.00,1180.0,5650.0,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209,538000.0,3.0,2.25,2570.0,7242.0,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225,180000.0,2.0,1.00,770.0,10000.0,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209,604000.0,4.0,3.00,1960.0,5000.0,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218,510000.0,3.0,2.00,1680.0,8080.0,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [62]:
houseCleanDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  float64
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  float64
 6   sqft_lot       21613 non-null  float64
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [63]:
''' Convert date column datatype from string to datetime and save to csv '''
import datetime as dt
houseCleanDF[['date']] = houseCleanDF[['date']].applymap(str).applymap(lambda s: '{}-{}-{}'.format(s[0:4],s[4:6],s[6:]))
houseCleanDF['quarter'] = pd.to_datetime(houseCleanDF['date']).dt.quarter
houseCleanDF.to_csv('./data/Modified/houseClean_dates.csv')
houseCleanDF.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,quarter
0,7129300520,2014-10-13,221900.0,3.0,1.00,1180.0,5650.0,1.0,0,0,...,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,4
1,6414100192,2014-12-09,538000.0,3.0,2.25,2570.0,7242.0,2.0,0,0,...,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,4
2,5631500400,2015-02-25,180000.0,2.0,1.00,770.0,10000.0,1.0,0,0,...,770,0,1933,0,98028,47.7379,-122.233,2720,8062,1
3,2487200875,2014-12-09,604000.0,4.0,3.00,1960.0,5000.0,1.0,0,0,...,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,4
4,1954400510,2015-02-18,510000.0,3.0,2.00,1680.0,8080.0,1.0,0,0,...,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,1


In [64]:
''' Subset of house_datesClean dataframe for columns: date, price, zipcode, sqft_lot '''
zipQuarterDF = houseCleanDF[['date','price','sqft_lot','zipcode','quarter']]
zipQuarterDF.head()

,date,price,sqft_lot,zipcode,quarter
0,2014-10-13,221900.0,5650.0,98178,4
1,2014-12-09,538000.0,7242.0,98125,4
2,2015-02-25,180000.0,10000.0,98028,1
3,2014-12-09,604000.0,5000.0,98136,4
4,2015-02-18,510000.0,8080.0,98074,1


In [65]:
zipQuarterDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      21613 non-null  object 
 1   price     21613 non-null  float64
 2   sqft_lot  21613 non-null  float64
 3   zipcode   21613 non-null  int64  
 4   quarter   21613 non-null  int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 844.4+ KB


In [66]:
''' Order zipcode data by date. '''
zipQuarterDF = zipQuarterDF.sort_values(by=['date']).reset_index(drop=True)
zipQuarterDF.head()

,date,price,sqft_lot,zipcode,quarter
0,2014-05-02,437500.0,35100.0,98027,2
1,2014-05-02,790000.0,4750.0,98117,2
2,2014-05-02,675000.0,67518.0,98029,2
3,2014-05-02,555000.0,6500.0,98065,2
4,2014-05-02,440000.0,8119.0,98006,2


In [67]:
''' Add lat-long columns '''
zipQuarterDF[['lat','long']] = houseCleanDF[['lat','long']]

In [68]:
''' Column of categorical data and save as csv.'''
# zipQuarterDF['quarter'] = pd.to_datetime(zipQuarterDF['date']).dt.quarter
zipQuarterDF.to_csv('./data/Modified/zip_by_quarter.csv')
zipQuarterDF.head()

,date,price,sqft_lot,zipcode,quarter,lat,long
0,2014-05-02,437500.0,35100.0,98027,2,47.5112,-122.257
1,2014-05-02,790000.0,4750.0,98117,2,47.7210,-122.319
2,2014-05-02,675000.0,67518.0,98029,2,47.7379,-122.233
3,2014-05-02,555000.0,6500.0,98065,2,47.5208,-122.393
4,2014-05-02,440000.0,8119.0,98006,2,47.6168,-122.045


In [69]:
''' Average lot size by zipcode '''
avglot_zip = pd.Series(round(zipQuarterDF.groupby(['zipcode', 'quarter'])['sqft_lot'].mean(), 2))
print(avglot_zip)

zipcode  quarter
98001    1          12117.47
         2          13190.66
         3          17561.97
         4          15853.81
98002    1           7299.55
                      ...   
98198    4          10425.67
98199    1           5284.74
         2           5379.57
         3           5715.66
         4           5411.64
Name: sqft_lot, Length: 280, dtype: float64
